In [ ]:
import warnings
warnings.filterwarnings('ignore')
import os
from pathlib import Path

import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

from util import get_device, clean_df, copy_data_to_device

import re

import torch
from transformers import (
    GPT2LMHeadModel,
    GPT2Tokenizer,
    TextDataset,
    DataCollatorForLanguageModeling,
    Trainer,
    TrainingArguments,
    TrainerCallback,
    EarlyStoppingCallback,
)
import evaluate

In [ ]:
df = pd.read_csv('geo-reviews-dataset-2023.csv')
df.head()

,address,name_ru,rating,rubrics,text
0,"Екатеринбург, ул. Московская / ул. Волгоградск...",Московский квартал,3.0,Жилой комплекс,Московский квартал 2.\nШумно : летом по ночам ...
1,"Московская область, Электросталь, проспект Лен...",Продукты Ермолино,5.0,Магазин продуктов;Продукты глубокой заморозки;...,"Замечательная сеть магазинов в общем, хороший ..."
2,"Краснодар, Прикубанский внутригородской округ,...",LimeFit,1.0,Фитнес-клуб,"Не знаю смутят ли кого-то данные правила, но я..."
3,"Санкт-Петербург, проспект Энгельса, 111, корп. 1",Snow-Express,4.0,Пункт проката;Прокат велосипедов;Сапсёрфинг,Хорошие условия аренды. \nДружелюбный персонал...
4,"Тверь, Волоколамский проспект, 39",Студия Beauty Brow,5.0,"Салон красоты;Визажисты, стилисты;Салон бровей...",Топ мастер Ангелина топ во всех смыслах ) Немн...


In [5]:
def clean_address(address):
    # Замена слешей, не связанных с цифрами, на пробелы
    address = re.sub(r"(?<!\d)/|(?!\d)/", " ", address)
    # Удаление лишних пробелов
    address = re.sub(r"\s+", " ", address)

    return address

def clean_text(text):
    # Замена переносов строк пробелами
    text = re.sub(r"[\\n\\r]+", " ", text)

    # Удалить HTML-теги
    text = re.sub(r"<[^>]+>", "", text)

    # Удалить специальные символы
    text = re.sub(r"[^\w\s,.!?()]+", "", text)

    return text

In [ ]:
df_cleaned = df.copy()
df_cleaned = clean_df(df_cleaned)

df_cleaned = df_cleaned[df_cleaned['rating'] > 0]

df_cleaned['text'] = df_cleaned['text'].apply(clean_text)
df_cleaned['address'] = df_cleaned['address'].apply(clean_address)
df_cleaned['name_ru'] = df_cleaned['name_ru'].str.strip()
df_cleaned['rubrics'] = df_cleaned['rubrics'].str.strip().str.lower()

In [ ]:
df_cleaned.info()

<class 'pandas.core.frame.DataFrame'>
Index: 499762 entries, 0 to 499999
Data columns (total 5 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   address  499762 non-null  object 
 1   name_ru  499762 non-null  object 
 2   rating   499762 non-null  float64
 3   rubrics  499762 non-null  object 
 4   text     499762 non-null  object 
dtypes: float64(1), object(4)
memory usage: 22.9+ MB


In [2]:
MODEL_PATH = 'data/model/'  # каталог модели
CACHE_PATH = 'data/model/model_cache'  # каталог кэша
MODEL_NAME = 'ai-forever/rugpt3small_based_on_gpt2'  # используемая модель
DATASETS_PATH = 'data/dataset'  # путь к каталогу датасетов
OUTPUT_MODEL_NAME = "fine_tuned_geo_reviews_model" # имя модели после обучения

In [3]:
# Создание директории для кэша
cache_dir = Path(CACHE_PATH)
os.makedirs(cache_dir, exist_ok=True)

tokenizer = GPT2Tokenizer.from_pretrained(MODEL_NAME, cache_dir=str(cache_dir))
model = GPT2LMHeadModel.from_pretrained(MODEL_NAME, cache_dir=str(cache_dir))

In [ ]:
df_X = df_cleaned.copy()

df_X['input'] = df_cleaned.apply(
    lambda
        row: f"<name_ru> {row['name_ru']} <rubrics> {row['rubrics']} <rating> {row['rating']} <address> {row['address']} {tokenizer.eos_token}",
    axis=1,
)
df_X['output'] = df_cleaned.apply(lambda row: f"<text> {row['text']} {tokenizer.eos_token}", axis=1)

In [ ]:
train_size=0.8
val_size=0.1
test_size=0.1

lines = [f'{input_text} {target_text}\n' for input_text, target_text in zip(df_X['input'], df_X['output'])]

# Разделение на тренировочный набор и набор для валидации и теста
train_lines, temp_lines = train_test_split(lines, train_size=train_size, random_state=42)
val_lines, test_lines = train_test_split(temp_lines, train_size=val_size / (val_size + test_size), random_state=42)

In [ ]:
# Создание директории для data
data_dir = Path(DATASETS_PATH)
os.makedirs(data_dir, exist_ok=True)

data_path = Path(DATASETS_PATH)
full_dataset_path = data_path.joinpath('full_dataset.txt')
train_dataset_path = data_path.joinpath('train_dataset.txt')
val_dataset_path = data_path.joinpath('val_dataset.txt')
test_dataset_path = data_path.joinpath('test_dataset.txt')

# Сохранение выборок в отдельные файлы
with open(full_dataset_path, 'w', encoding='utf-8') as f:
    f.writelines(lines)
with open(train_dataset_path, 'w', encoding='utf-8') as f:
    f.writelines(train_lines)
with open(val_dataset_path, 'w', encoding='utf-8') as f:
    f.writelines(val_lines)
with open(test_dataset_path, 'w', encoding='utf-8') as f:
    f.writelines(test_lines)

In [ ]:
train_dataset = TextDataset(tokenizer=tokenizer, file_path=str(full_dataset_path), block_size=256)
eval_dataset = TextDataset(tokenizer=tokenizer, file_path=str(val_dataset_path), block_size=256)

data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)

In [ ]:
# Класс для кастомного колбэка в процессе обучения
class TrainingCallback(TrainerCallback):
     def on_log(self, args, state, control, logs=None, **kwargs):
         if logs:
            if "eval_loss" in logs:
                eval_loss = logs["eval_loss"]
                print("Evaluation", "eval_loss", eval_loss)
                perplexity = np.exp(eval_loss)
                logs["eval_perplexity"] = perplexity
                print("Evaluation", "Perplexity", perplexity)
                


def fine_tune(train_dataset, eval_dataset, output_name=OUTPUT_MODEL_NAME, num_train_epochs=5,
                  per_device_train_batch_size=16, learning_rate=5e-5, save_steps=10_000):
    """Процесс дообучения модели на кастомных данных."""

    model_path = Path(MODEL_PATH)

    training_args = TrainingArguments(
        output_dir=str(model_path.joinpath(output_name)),
        overwrite_output_dir=True,
        num_train_epochs=num_train_epochs,
        per_device_train_batch_size=per_device_train_batch_size,
        save_steps=save_steps,
        learning_rate=learning_rate,
        save_total_limit=2,
        eval_steps=1000,
        eval_strategy='epoch',
        save_strategy='epoch',
        load_best_model_at_end=True,
        no_cuda=not torch.cuda.is_available(),
        fp16=True,
        warmup_steps=5000,
        lr_scheduler_type='linear',
        metric_for_best_model="eval_loss",
        weight_decay=0.01,
        save_safetensors=False
    )

    # Настройка Trainer
    trainer = Trainer(
        model=model,
        args=training_args,
        data_collator=data_collator,
        train_dataset=train_dataset,
        eval_dataset=eval_dataset,
        callbacks=[TrainingCallback(), EarlyStoppingCallback(early_stopping_patience=3)],
    )

    trainer.train()

    test_dataset = TextDataset(tokenizer=tokenizer, file_path=str(test_dataset_path), block_size=256)
    test_results = trainer.evaluate(test_dataset)

    print(f'Validation eval_loss: {test_results["eval_loss"]}')

    model.save_pretrained(str(model_path.joinpath(output_name)))
    tokenizer.save_pretrained(str(model_path.joinpath(output_name)))


In [ ]:
# Запуск обучения модели
fine_tune(
    train_dataset, 
    eval_dataset,
    output_name=OUTPUT_MODEL_NAME,
    num_train_epochs=1,
    per_device_train_batch_size=16,
    learning_rate=5e-5,
)

Epoch,Training Loss,Validation Loss,Perplexity
1,2.215000,2.205626,9.075931


Evaluation eval_loss 2.2056260108947754
Evaluation Perplexity 9.075931421795977


There were missing keys in the checkpoint model loaded: ['lm_head.weight'].


Evaluation eval_loss 2.205807685852051
Evaluation Perplexity 9.07758044103748
eval_loss: 2.205807685852051


In [ ]:
device = get_device()
model.to(device)

def prepare_input_text(name_ru, rubrics, rating, address=None):
    address_part = f"<address> {address} " if address else ""
    return f"<name_ru> {name_ru} <rubrics> {rubrics} {address_part}<rating> {rating} {tokenizer.eos_token}"

def generate_text(input_text, max_length=135, temperature=0.9, top_k=50, top_p=0.95, no_repeat_ngram_size=2):
    model.eval()
    inputs = tokenizer(input_text, return_tensors="pt").to(device)
    data = copy_data_to_device(inputs["input_ids"], device)

    with torch.no_grad():
        outputs = model.generate(
            data,
            max_length=max_length,
            temperature=temperature,
            top_k=top_k,
            top_p=top_p,
            eos_token_id=tokenizer.eos_token_id,
            no_repeat_ngram_size=no_repeat_ngram_size,
            do_sample=True
        )

    return tokenizer.decode(outputs[0], skip_special_tokens=True)

In [6]:
model_name = OUTPUT_MODEL_NAME
tokenizer = GPT2Tokenizer.from_pretrained(f'{MODEL_PATH}{model_name}', cache_dir=CACHE_PATH)
model = GPT2LMHeadModel.from_pretrained(f'{MODEL_PATH}{model_name}', cache_dir=CACHE_PATH)

In [9]:
name_ru = 'LimeFit'
rubrics = 'Фитнес-клуб'
rating = '3'
address = 'Москва, 1-я улица Соколиной Горы, 152, стр. 2'

input_text = prepare_input_text(name_ru, rubrics, rating, address)
generated_text = generate_text(input_text)
generated_text

'<name_ru> LimeFit <rubrics> Фитнес-клуб <address> Москва, 1-я улица Соколиной Горы, 152, стр. 2 <rating> 3  <text> Отличное место, очень хорошая атмосфера,  приятная музыка, отличный тренерский состав, приятная, располагающая атмосфера.  Очень много занятий, так же есть зал для групповых и индивидуальных тренировок, и в общем все, что бы можно было улучшить, если чтото не так.\n \n<<b> MIUZ diamonds <meaning> ювелирный магазин;магазин часов;салон'

In [ ]:
index = generated_text.find('<text>') + 6
text = generated_text[index:]
text

' Отличный фитнес клуб! Тренеры супер профессионалы! Все время проходят интересные мероприятия! Спасибо тренерам за терпение и внимательность!\n <texnosfera.ru <nurl> Кератиновое выпрямление и депиляция;салон красоты;косметология <raspectus> солярий <it-компания <robot> салон красоты <arbmotion'

In [ ]:
name_ru = 'Snow-Express'
rubrics = 'Фитнес-клуб'
rating = '1'
address = 'Москва, 1-я улица Соколиной Горы, 2'

input_text = prepare_input_text(name_ru, rubrics, rating, address)
generated_text = generate_text(input_text)
generated_text

'<name_ru> Snow-Express <rubrics> Фитнес-клуб <address> Москва, 1-я улица Соколиной Горы, 2 <rating> 1  <text> Отличное место. Есть разные направления. Мне нравится.  Всегда чисто. Персонал отзывчивый. Всегда помогут найти то, что нужно, подскажут, если чтото не получается в приоритете. Я очень довольна, есть с чем сравнить. Обязательно вернусь сюда, и не один раз.\n <<textt> Уютно и чисто, удобное расположение. Приятный приветливый персонал. Очень часто сюда прихожу.'

In [ ]:
index = generated_text.find('<text>') + 6
text = generated_text[index:]
text

' Отличное место. Есть разные направления. Мне нравится.  Всегда чисто. Персонал отзывчивый. Всегда помогут найти то, что нужно, подскажут, если чтото не получается в приоритете. Я очень довольна, есть с чем сравнить. Обязательно вернусь сюда, и не один раз.\n <<textt> Уютно и чисто, удобное расположение. Приятный приветливый персонал. Очень часто сюда прихожу.'

In [11]:
# Функция очистки текста
def clean_text(text):

    # Удаление латиницы
    text = re.sub(r'[A-Za-z]', '', text)

    return text

filtered_text = re.sub(r"<.*?>", "", text).strip()
filtered_text = clean_text(filtered_text)
filtered_text

'Ужасная организация. Если я вам нахамила, то я готова вас слушать. Записали на удобное для вас время  к 930. Не дождавшись, ушли, так и не дождавшись. На вопрос, что за квест и кто будет?  ответили,что не знают,  пока не попросили о нем. А вот если я сама виновата, и просто не заметила, как меня забросили, могу ли я вернуть деньги'

In [12]:
cases = [{
    'name_ru': 'Snow-Express',
    'rubrics': 'Фитнес-клуб',
    'rating': '1.0',
    'address': 'Москва, 1-я улица Соколиной Горы, 2',
    },{
    'name_ru': 'Snow-Express',
    'rubrics': 'Фитнес-клуб',
    'rating': '5.0',
    'address': 'Москва, 1-я улица Соколиной Горы, 2',
    },{
    'name_ru': 'Beauty',
    'rubrics': 'салон',
    'rating': '5.0',
    'address': 'Екатеринбург, ул. Московская ул.',
    }
]

predictions = []

for texts in cases:
    input_text = prepare_input_text(texts['name_ru'], texts['rubrics'], texts['rating'], texts['address'])
    generated_text = generate_text(input_text)
    index = generated_text.find('<text>') + 6
    text = generated_text[index:]
    filtered_text = re.sub(r"<.*?>", "", text).strip()
    filtered_text = clean_text(filtered_text)
    predictions.append(filtered_text)

references = [
    "Это было полное разочарование. Тренажеры старые и многие сломаны, вентиляция не работает, в зале душно и неприятный запах. Персонал абсолютно равнодушный, никто не помогает и не следит за порядком. За такие деньги ожидал совершенно другого уровня сервиса. Это худший фитнес-клуб, в котором я когда-либо был. Не тратьте свое время и деньги.",
    "Очень доволен качеством услуг. Зал просторный, оборудование современное и всегда в рабочем состоянии. Особенно хочу отметить профессионализм тренеров — они настоящие мастера своего дела, всегда подскажут и помогут составить программу тренировок. Атмосфера дружелюбная и мотивирующая. Цены адекватные. Однозначно рекомендую!",
    "Это место, где можно по-настоящему расслабиться и получить качественные услуги. Делала маникюр, стрижку и уход за лицом — результат всегда превосходный. Мастера очень внимательные, аккуратные и всегда учитывают все пожелания клиента. В салоне чисто, уютно, играет приятная музыка. Сервис на высшем уровне. Спасибо за красоту и отличное настроение!"
]


In [13]:
predictions

['Раньше все было супер, сейчас ужас!! Останавливались в этом отеле на новогодние праздники, на территории отеля грязно, нет кондиционера,  уборка  отвратительная, все окна грязные и окна закрыты, не закрываются, в душевой воде постоянно какие то камни. Не рекомендую \n \n студия перманентного макияжа',
 'Самый лучший клуб в районе! Обслуживание топ, тренер всегда поддержит и предложит отличный совет. Я с удовольствием хожу и не жалею, что нашла этот клуб! \n \n салон красоты  ногтевая студия  эпиляция;салон бровей и ресниц  маник',
 'Хожу туда на стрижку и окрашивание. Очень приветливый персонал, всегда расскажут, посоветуют. Идут на встречу если чтото не получается, я делаю мелирование и укладку.  Очень нравится стрижка у мастера Лены, результат шикарный!\n  Отличный салон. Стильные, стильные модели. Много различных занятий для деток. Мастера очень внимательные. В салоне очень чисто и уютно. Ходу с удовольствием!']

In [15]:
# загружаем метрики через evaluate
bleu = evaluate.load("bleu")

In [16]:
bleu_score = bleu.compute(predictions=predictions, references=references)
print("BLEU:", bleu_score)

BLEU: {'bleu': 0.03529851938606751, 'precisions': [0.2795031055900621, 0.05063291139240506, 0.01935483870967742, 0.006578947368421052], 'brevity_penalty': 0.96341879037603, 'length_ratio': 0.9640718562874252, 'translation_length': 161, 'reference_length': 167}
